---
Following the Lynch et al paper https://iopscience.iop.org/article/10.3847/2041-8213/aacf9f

"We find that the gravitational-wave alert threshold change that would roughly double the number of detectable astrophysical events would increase the false-alarm rate by more than 5 orders of magnitude from 1 per 100 years to more than 1000 per year."

"We argue that EM follow-up thresholds for low-significance candidates should be set on the basis of alert purity (Pastro) and not false-alarm rate."

$FAR = FAR_8 \times [-\frac{\rho - 8}{\alpha}]$

* $FAR_8$ is the false alarm rate at a network ranking statistic $\rho = 8$
* $\alpha$ is a scale parameter that determines the steepness of the fall off

---
From Nitz et al. 2017 https://doi.org/10.3847/1538-4357/aa8f50

The SNR and signal consistency tests (Allen paper) result in the detection statistic.

"We present a new detection statistic that reduces the false alarm rate of the search across the entire target signal space: binary neutron stars, binary black holes, and neutron star–black hole binaries. This detection statistic uses information from both the physical parameters of the gravitational-wave signal and the expected distribution of the noise background to rescale the “combined effective signal-to-noise ratio” used previously by the PyCBC search."

"The detection statistic used in Advanced LIGO’s first observing run was the quadrature sum of the re-weighted signal-to-noise ratio $\hat{\rho}$ observed in each detector, denoted $\hat{\rho_c}$"

$\hat{\rho} = \frac{\rho}{[1+(\chi^2)^3]^{1/6}}$

"The non-stationarity of the detectors’ noise means that it is impossible to construct an analytic model of the search’s false alarm rate."

They look like they use PyCBC data to estimate the FAR for the run.

---

### Questions
* How do I determine my own $\alpha$ ?
* How does $\rho$ relate to the optimal SNR?
* How do you detemrine $FAR_8$?

### To do:
* look for examples of FAR estimation for observing runs
    * probably linked in LIGO papers